In [30]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report as cr
import pickle

warnings.simplefilter(action = 'ignore', category = FutureWarning)
k_folds = KFold(n_splits = 10)

In [31]:
dataset = pd.read_csv("IMDB_dataset_preprocessed.csv")
dataset.drop(dataset.tail(45000).index, inplace = True)
reviews = [w for w in dataset["review"]]

In [32]:
pickle_in = open("bert_vectors_preprocessed1.pickle", "rb")
word_vectors1 = pickle.load(pickle_in)

In [33]:
pickle_in = open("bert_vectors_preprocessed2.pickle", "rb")
word_vectors2 = pickle.load(pickle_in)

In [34]:
pickle_in = open("bert_vectors_preprocessed3.pickle", "rb")
word_vectors3 = pickle.load(pickle_in)

In [35]:
pickle_in = open("bert_vectors_preprocessed4.pickle", "rb")
word_vectors4 = pickle.load(pickle_in)

In [36]:
pickle_in = open("bert_vectors_preprocessed5.pickle", "rb")
word_vectors5 = pickle.load(pickle_in)

In [37]:
word_vectors_concatenated = np.concatenate((word_vectors1, word_vectors2, word_vectors3, word_vectors4, word_vectors5))

In [38]:
def getReviewVector(review):
    return np.mean([word for word in review], axis = 0)

In [39]:
x = []
for i in range(len(word_vectors_concatenated)):
    x.append(getReviewVector(word_vectors_concatenated[i]))

In [40]:
y = pd.get_dummies(dataset["sentiment"])
y = y.iloc[:,1].values

In [41]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [42]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [43]:
score = accuracy_score(y_test, y_pred)
print(score)
print(cr(y_test, y_pred))

0.743
              precision    recall  f1-score   support

           0       0.76      0.75      0.75       525
           1       0.73      0.74      0.73       475

    accuracy                           0.74      1000
   macro avg       0.74      0.74      0.74      1000
weighted avg       0.74      0.74      0.74      1000



In [44]:
# 10-fold cross-validation
scores = cross_val_score(rfc, x, y, cv = k_folds)

In [45]:
# 10-fold cross-validation results
print("CV scores: ", scores)
print("Avg CV score: ", scores.mean())

CV scores:  [0.768 0.768 0.728 0.75  0.722 0.732 0.722 0.746 0.742 0.746]
Avg CV score:  0.7424
